# Table 1 and 2 generation

### Import and configure

In [1]:
from sas_pet.utils import photon_shards
from sas_pet.analysis.fates import DetectorCylinder, accumulate_photon_fates, validate_fates

MC_DATA_FOLDER = "../results/MC_data_1e7"
Rs = [5.0, 10.0, 15.0]

det = DetectorCylinder(radius_cm=15.0, ring_length_cm=20.0)

stats_by_R = {}
warn_by_R = {}

for R in Rs:
    shards = photon_shards(MC_DATA_FOLDER, R)
    if not shards:
        raise FileNotFoundError(f"No photon shards for R={R} in {MC_DATA_FOLDER}")

    stats = accumulate_photon_fates(shards, det=det)
    stats_by_R[float(R)] = stats
    warn_by_R[float(R)] = validate_fates(stats)

warn_by_R

{5.0: [], 10.0: [], 15.0: []}

### Build display tables

In [15]:
import numpy as np
import pandas as pd

def pct(n: int, denom: int) -> float:
    return (100.0 * float(n) / float(denom)) if denom else float("nan")

def cell_txt(n: int, denom: int) -> str:
    return f"{int(n):,} ({pct(n, denom):.1f}%)"

rows_emitted = [
    ("Emitted", "emitted", "emitted"),
    ("Absorbed (in object)", "absorbed", "emitted"),
    ("Exited object", "exited", "emitted"),
]

rows_exited = [
    ("Exited object", "exited", "exited"),
    ("Exited, not scattered", "not_scattered", "exited"),
    ("Exited, scattered", "scattered", "exited"),
    (rf"Exited, hit detector ", "hit_detector", "exited"),
    ("Exited, missed detector", "missed_detector", "exited"),
]

def make_df(rows_spec):
    rows = []
    for label, key, denom_key in rows_spec:
        row = {"Category": label}
        for R in Rs:
            s = stats_by_R[float(R)]
            denom = int(s[denom_key])
            if key == denom_key:
                row[f"R={R:.0f} cm"] = f"{int(s[key]):,} (100.0%)"
            else:
                row[f"R={R:.0f} cm"] = cell_txt(int(s[key]), denom)
        rows.append(row)
    return pd.DataFrame(rows)

df_emitted = make_df(rows_emitted)
df_exited = make_df(rows_exited)

In [16]:
df_emitted

,Category,R=5 cm,R=10 cm,R=15 cm
0,Emitted,"20,000,000 (100.0%)","20,000,000 (100.0%)","20,000,000 (100.0%)"
1,Absorbed (in object),"68,829 (0.3%)","619,991 (3.1%)","1,752,206 (8.8%)"
2,Exited object,"19,931,171 (99.7%)","19,380,009 (96.9%)","18,247,794 (91.2%)"


In [17]:
df_exited

,Category,R=5 cm,R=10 cm,R=15 cm
0,Exited object,"19,931,171 (100.0%)","19,380,009 (100.0%)","18,247,794 (100.0%)"
1,"Exited, not scattered","10,494,265 (52.7%)","6,130,763 (31.6%)","3,960,534 (21.7%)"
2,"Exited, scattered","9,436,906 (47.3%)","13,249,246 (68.4%)","14,287,260 (78.3%)"
3,"Exited, hit detector","10,594,370 (53.2%)","8,045,072 (41.5%)","3,435,008 (18.8%)"
4,"Exited, missed detector","9,336,801 (46.8%)","11,334,937 (58.5%)","14,812,786 (81.2%)"


### Make LaTex strings

In [6]:
def latex_int(n: int) -> str:
    return f"{int(n):,}".replace(",", r"\,")

def latex_pct(x: float) -> str:
    return f"{x:.1f}" + r"\%"

def cell_latex(n: int, denom: int) -> str:
    return f"{latex_int(n)} ({latex_pct(pct(n, denom))})"

def print_latex_table(rows_spec, denom_key: str):
    header = "Category & " + " & ".join([rf"$R={R:.0f}\,\mathrm{{cm}}$" for R in Rs]) + r" \\"
    print(header)
    print(r"\hline")
    for label, key, _ in rows_spec:
        cells = []
        for R in Rs:
            s = stats_by_R[float(R)]
            denom = int(s[denom_key])
            if key == denom_key:
                cells.append(f"{latex_int(int(s[key]))} (100.0\\%)")
            else:
                cells.append(cell_latex(int(s[key]), denom))
        print(f"{label} & " + " & ".join(cells) + r" \\")

In [7]:
print_latex_table(rows_emitted, denom_key="emitted")

Category & $R=5\,\mathrm{cm}$ & $R=10\,\mathrm{cm}$ & $R=15\,\mathrm{cm}$ \\
\hline
Emitted & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) \\
Absorbed (in object) & 68\,829 (0.3\%) & 619\,991 (3.1\%) & 1\,752\,206 (8.8\%) \\
Exited object & 19\,931\,171 (99.7\%) & 19\,380\,009 (96.9\%) & 18\,247\,794 (91.2\%) \\


In [8]:
print_latex_table(rows_exited, denom_key="exited")

Category & $R=5\,\mathrm{cm}$ & $R=10\,\mathrm{cm}$ & $R=15\,\mathrm{cm}$ \\
\hline
Exited object & 19\,931\,171 (100.0\%) & 19\,380\,009 (100.0\%) & 18\,247\,794 (100.0\%) \\
Exited, not scattered & 10\,494\,265 (52.7\%) & 6\,130\,763 (31.6\%) & 3\,960\,534 (21.7\%) \\
Exited, scattered & 9\,436\,906 (47.3\%) & 13\,249\,246 (68.4\%) & 14\,287\,260 (78.3\%) \\
Exited, hit detector ($R_\mathrm{det}=15\,\mathrm{cm}$, $L=20\,\mathrm{cm}$) & 10\,594\,370 (53.2\%) & 8\,045\,072 (41.5\%) & 3\,435\,008 (18.8\%) \\
Exited, missed detector & 9\,336\,801 (46.8\%) & 11\,334\,937 (58.5\%) & 14\,812\,786 (81.2\%) \\
